In [4]:
import json
import requests
import websockets
import sys
from typing import Tuple, List
from math import log
inf = sys.maxsize

# Global Variables and Functions

In [5]:
rates = [
                    {
                        "ETH": {"ETH": 1, "BTC": inf,"XMR": inf},
                        "BTC": {"ETH": inf,"BTC": 1,"XMR": inf},
                        "XMR": {"ETH": inf,"BTC": inf, "XMR": 1}
                    }
]

currencies = (
    'ETH', 
    'BTC', 
    'XMR'
)

def negate_logarithm_convertor(graph: Tuple[Tuple[float]]) -> List[List[float]]:
    ''' log of each rate in graph and negate it'''
    result = [[-log(edge) for edge in row.values()] for row in graph[0].values()]
    return result


def arbitrage(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    ''' Calculates arbitrage situations and prints out the details of this calculations'''

    trans_graph = negate_logarithm_convertor(rates_matrix)

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and get the right result

    source = 0
    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n
    
    min_dist[source] = source

    # 'Relax edges |V-1| times'
    for _ in range(n-1):
        for source_curr in range(n):
            for dest_curr in range(n):
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                    pre[dest_curr] = source_curr

    # if we can still relax edges, then we have a negative cycle
    for source_curr in range(n):
        for dest_curr in range(n):
            if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                # negative cycle exists, and use the predecessor chain to print the cycle
                print_cycle = [dest_curr, source_curr]
                # Start from the source and go backwards until you see the source vertex again or any vertex that already exists in print_cycle array
                while pre[source_curr] not in  print_cycle:
                    print_cycle.append(pre[source_curr])
                    source_curr = pre[source_curr]
                print_cycle.append(pre[source_curr])
                print("Arbitrage Opportunity: \n")
                print(" --> ".join([currencies[p] for p in print_cycle[::-1]]))

# Apply public connect token

In [6]:
url = 'https://api.kucoin.com/api/v1/bullet-public'

In [7]:
x = requests.post(url)

In [8]:
cjson = json.loads(x.text)

In [9]:
endpoint = cjson['data']['instanceServers'][0]['endpoint']
token = cjson['data']['token']

# Multiplex Websocket Connection

In [10]:
multi = {"id": "1Jpg30DEdU", "type": "openTunnel", "newTunnelId": "bt1", "response": False}
# command used to open tunnel

In [11]:
multi_ETHBTC = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:ETH-BTC",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [12]:
multi_XMRBTC = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:XMR-BTC",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [13]:
multi_XMRETH = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:XMR-ETH",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [14]:
async with websockets.connect(endpoint+"?token="+token) as conn:
    await conn.send(json.dumps(multi))  # initial open tunel command
    res = await conn.recv()
    
    await conn.send(json.dumps(multi_ETHBTC)) # send tunnel subscription
    res = await conn.recv()
    await conn.send(json.dumps(multi_XMRBTC))  # send tunnel subscription
    res = await conn.recv()
    await conn.send(json.dumps(multi_XMRETH))  # send tunnel subscription
    res = await conn.recv()
    
    counter = 0
    spotPrice = 0
    futurePrice = 0
    while True:
        
        res = await conn.recv()
        vals = json.loads(res)

        if (vals['topic']) == '/market/ticker:ETH-BTC':
            rates[0]['ETH']['BTC'] = 1 / float(vals['data']['bestAsk'])
            rates[0]['BTC']['ETH'] = float(vals['data']['bestAsk'])

        elif (vals['topic']) == '/market/ticker:XMR-ETH':
            rates[0]['XMR']['ETH'] = 1 / float(vals['data']['bestAsk'])
            rates[0]['ETH']['XMR'] = float(vals['data']['bestAsk'])
        
        elif (vals['topic']) == '/market/ticker:XMR-BTC':
            rates[0]['XMR']['BTC'] = 1 / float(vals['data']['bestAsk'])
            rates[0]['BTC']['XMR'] = float(vals['data']['bestAsk'])
        
        
        if __name__ == "__main__":
            arbitrage(currencies, rates)
        counter += 1
    eth_tunnel = {"id": "1JpoPamgFM", "type": "closeTunnel", "tunnelId": "bt1", "response": False}            
    btc_tunnel = {"id": "1JpoPamgFN", "type": "closeTunnel", "tunnelId": "bt1", "response": False}
    await conn.send(json.dumps(eth_tunnel))
    await conn.send(json.dumps(btc_tunnel))
    await conn.close()

Arbitrage Opportunity: 

BTC --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

XMR --> BTC --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

ETH --> XMR --> ETH --> BTC
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
A

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

ETH --> XMR --> BTC --> ETH
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

...

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH --> XMR
Arbitrage Opportunity: 

XMR --> BTC --> ETH -

CancelledError: 

In [15]:
result = [[edge for edge in row.values()] for row in rates[0].values()]
result

[[1, 14.981946754161235, 0.09485],
 [0.066747, 1, 0.006322],
 [10.542962572482868, 158.17779183802594, 1]]